In [1]:
import pandas as pd
import json 

In [2]:
prompt_results_df = pd.read_csv('C:/Users/neals/Desktop/practicum/prompt_results_classification.csv')
prompt_results_df['prompt'] = prompt_results_df['prompt'].fillna('')
prompt_results_df.head()

,image_id,image_file,image_path,prompt,generated_text,category,processing_time
0,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,,an aerial view of a parking lot with several y...,1,2.130118
1,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,"Question: is there a plane in this image, If y...",yes,1,1.203270
2,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: is anything moving? Answer:,yes,1,0.766499
3,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: Is there a car? Answer:,yes,1,0.862881
4,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: How many cars are there? Answer:,1,1,0.788784


In [3]:
print(prompt_results_df['prompt'].unique())

[''
 "Question: is there a plane in this image, If you are not sure about the answer, say you don't know? Answer:"
 'Question: is anything moving? Answer:'
 'Question: Is there a car? Answer:'
 'Question: How many cars are there? Answer:'
 'Question: Is there a truck? Answer:'
 'Question: How many trucks are there? Answer:'
 'Question: Is there a ship? Answer:'
 'Question: How many ships are there? Answer:'
 'Question: Is there a plane? Answer:'
 'Question: How many planes are there? Answer:'
 'Question: Is there a storage-tank? Answer:'
 'Question: How many storage-tanks are there? Answer:'
 'Question: Is there a swimming-pool? Answer:'
 'Question: How many swimming-pools are there? Answer:'
 'Question: Is there a harbor? Answer:'
 'Question: How many harbors are there? Answer:']


In [16]:
processed_time_df = prompt_results_df.groupby('prompt')['processing_time'].mean().reset_index()
processed_time_df

,prompt,processing_time
0,,1.181769
1,Question: How many cars are there? Answer:,0.561111
2,Question: How many harbors are there? Answer:,0.730949
3,Question: How many planes are there? Answer:,0.515615
4,Question: How many ships are there? Answer:,0.601957
5,Question: How many storage-tanks are there? An...,0.669958
6,Question: How many swimming-pools are there? A...,0.389590
7,Question: How many trucks are there? Answer:,0.468850
8,Question: Is there a car? Answer:,0.775663
9,Question: Is there a harbor? Answer:,0.818329


In [4]:
with open('E:/datasets/DOTA_dataset_512/val_coco.json') as f:
    gt_json = json.load(f)

category_map = {cat['id']:cat['name'] for cat in gt_json['categories']}
annotation_df = pd.DataFrame(gt_json['annotations'])
annotation_df['category'] = annotation_df['category_id'].map(category_map)
#annotation_df.head()

,id,image_id,category_id,bbox,area,iscrowd,segmentation,polygon,category
0,1,1,2,"[488.0, 493.0, 23.0, 18.0]",403.128880,0,"[[511.0, 511.0, 511.0, 493.4035087719298, 488....","POLYGON ((511 511, 511 493.4035087719298, 488 ...",large-vehicle
1,2,1,2,"[477.0, 464.0, 34.0, 23.758620689655174]",729.344828,0,"[[511.0, 466.2758620689655, 478.0, 464.0, 477....","POLYGON ((511 466.2758620689655, 478 464, 477 ...",large-vehicle
2,3,1,2,"[478.0, 236.0, 33.0, 105.0]",2007.500000,0,"[[511.0, 287.3333333333333, 500.0, 236.0, 478....","POLYGON ((511 287.3333333333333, 500 236, 478 ...",large-vehicle
3,4,1,2,"[382.0, 253.0, 40.0, 103.0]",2105.500000,0,"[[402.0, 253.0, 382.0, 258.0, 401.0, 356.0, 42...","POLYGON ((402 253, 382 258, 401 356, 422 349, ...",large-vehicle
4,5,1,2,"[356.0, 263.0, 40.0, 103.0]",2015.500000,0,"[[374.0, 263.0, 356.0, 269.0, 375.0, 366.0, 39...","POLYGON ((374 263, 356 269, 375 366, 396 358, ...",large-vehicle


In [5]:
category_agg = annotation_df.groupby('image_id')['category'].agg(lambda x: list(set(x))).reset_index()
category_agg.rename(columns={'category': 'category_list'}, inplace=True)
prompt_results_df = prompt_results_df.merge(category_agg, on='image_id', how='left')
#prompt_results_df.head(10)


,image_id,image_file,image_path,prompt,generated_text,category,processing_time,category_list
0,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,,an aerial view of a parking lot with several y...,1,2.130118,[large-vehicle]
1,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,"Question: is there a plane in this image, If y...",yes,1,1.203270,[large-vehicle]
2,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: is anything moving? Answer:,yes,1,0.766499,[large-vehicle]
3,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: Is there a car? Answer:,yes,1,0.862881,[large-vehicle]
4,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: How many cars are there? Answer:,1,1,0.788784,[large-vehicle]
5,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: Is there a truck? Answer:,yes,1,0.627445,[large-vehicle]
6,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: How many trucks are there? Answer:,1,1,0.337862,[large-vehicle]
7,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: Is there a ship? Answer:,yes,1,0.698961,[large-vehicle]
8,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: How many ships are there? Answer:,1,1,0.553368,[large-vehicle]
9,1,P0003_0.png,E:/datasets/DOTA_dataset_512/val/images/P0003_...,Question: Is there a plane? Answer:,yes,1,0.767955,[large-vehicle]


In [6]:
def correct_class_found(row, class_name):

    cats = row.category_list
    gen_text = row.generated_text

    if class_name in cats and gen_text.lower()=='yes': 
        return 1
    elif gen_text.lower() !='yes': 
        return 1
    #print(cats)
    #print(gen_text)
    return 0

def correct_vehicle_found(row):

    cats = row.category_list
    gen_text = row.generated_text
    vehicle = 'small-vehicle'in cats or 'large-vehicle' in cats
    if vehicle and gen_text.lower()=='yes': 
        return 1
    elif gen_text.lower() != 'yes':
        return 1

    return 0

In [7]:
pr_car = prompt_results_df[prompt_results_df['prompt']=='Question: Is there a car? Answer:']
#pr_car['correct'] = pr_car.apply(lambda x: correct_class_found(x,'small-vehicle'), axis=1 )
pr_car['correct'] = pr_car.apply(lambda x: correct_vehicle_found(x), axis=1 )

print(f"{pr_car['correct'].sum()}/{len(pr_car.index)} {100*pr_car['correct'].sum()/len(pr_car.index):.2f}%")

4678/7750 60.36%


C:\Users\neals\AppData\Local\Temp\ipykernel_32604\2199036655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_car['correct'] = pr_car.apply(lambda x: correct_vehicle_found(x), axis=1 )


In [8]:
pr_truck = prompt_results_df[prompt_results_df['prompt']=='Question: Is there a truck? Answer:']
pr_truck['correct'] = pr_truck.apply(lambda x: correct_class_found(x,'large-vehicle'), axis=1 )
#pr_truck['correct'] = pr_truck.apply(lambda x: correct_vehicle_found(x), axis=1 )

print(f"{pr_truck['correct'].sum()}/{len(pr_truck.index)} {100*pr_truck['correct'].sum()/len(pr_truck.index):.2f}%")

['small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['harbor', 'ship']
yes
['harbor']
yes
['harbor']
yes
['harbor']
yes
['harbor', 'ship']
yes
['harbor']
yes
['harbor']
yes
['harbor']
yes
['harbor']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['harbor']
yes
['harbor', 'ship']
yes
['harbor']
Yes
['harbor', 'ship']
yes
['harbor']
Yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['harbor']
Yes
['harbor', 'ship']
yes
['harbor']
yes
['harbor']
yes
['harbor']
yes
['harbor']
yes
['harbor']
yes
['harbor']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['harbor']
yes
['harbor']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['harbor', 'ship']
yes
['harbor', 'sh

C:\Users\neals\AppData\Local\Temp\ipykernel_32604\951196044.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_truck['correct'] = pr_truck.apply(lambda x: correct_class_found(x,'large-vehicle'), axis=1 )


In [9]:
pr_plane = prompt_results_df[prompt_results_df['prompt']=='Question: Is there a plane? Answer:']
pr_plane['correct'] = pr_plane.apply(lambda x: correct_class_found(x,'plane'), axis=1 )

print(f"{pr_plane['correct'].sum()}/{len(pr_plane.index)} {100*pr_plane['correct'].sum()/len(pr_plane.index):.2f}%")

['large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-ve

C:\Users\neals\AppData\Local\Temp\ipykernel_32604\3863743411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_plane['correct'] = pr_plane.apply(lambda x: correct_class_found(x,'plane'), axis=1 )


In [10]:
pr_plane = prompt_results_df[prompt_results_df['prompt']== "Question: is there a plane in this image, If you are not sure about the answer, say you don't know? Answer:"]
pr_plane['correct'] = pr_plane.apply(lambda x: correct_class_found(x,'plane'), axis=1 )

print(f"{pr_plane['correct'].sum()}/{len(pr_plane.index)} {100*pr_plane['correct'].sum()/len(pr_plane.index):.2f}%")

['large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-ve

C:\Users\neals\AppData\Local\Temp\ipykernel_32604\1736850440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_plane['correct'] = pr_plane.apply(lambda x: correct_class_found(x,'plane'), axis=1 )


In [11]:
pr_tank = prompt_results_df[prompt_results_df['prompt']== 'Question: Is there a storage-tank? Answer:']
pr_tank['correct'] = pr_tank.apply(lambda x: correct_class_found(x,'storage-tank'), axis=1 )

print(f"{pr_tank['correct'].sum()}/{len(pr_tank.index)} {100*pr_tank['correct'].sum()/len(pr_tank.index):.2f}%")

['large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-ve

C:\Users\neals\AppData\Local\Temp\ipykernel_32604\1763781353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_tank['correct'] = pr_tank.apply(lambda x: correct_class_found(x,'storage-tank'), axis=1 )


In [12]:
pr_pool = prompt_results_df[prompt_results_df['prompt']== 'Question: Is there a swimming-pool? Answer:']
pr_pool['correct'] = pr_pool.apply(lambda x: correct_class_found(x,'swimming-pool'), axis=1 )

print(f"{pr_pool['correct'].sum()}/{len(pr_pool.index)} {100*pr_pool['correct'].sum()/len(pr_pool.index):.2f}%")

['large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-ve

C:\Users\neals\AppData\Local\Temp\ipykernel_32604\3082335608.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_pool['correct'] = pr_pool.apply(lambda x: correct_class_found(x,'swimming-pool'), axis=1 )


In [13]:
pr_harbor = prompt_results_df[prompt_results_df['prompt']== 'Question: Is there a harbor? Answer:']
pr_harbor['correct'] = pr_harbor.apply(lambda x: correct_class_found(x,'harbor'), axis=1 )

print(f"{pr_harbor['correct'].sum()}/{len(pr_harbor.index)} {100*pr_harbor['correct'].sum()/len(pr_harbor.index):.2f}%")

['large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-ve

C:\Users\neals\AppData\Local\Temp\ipykernel_32604\2306431779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_harbor['correct'] = pr_harbor.apply(lambda x: correct_class_found(x,'harbor'), axis=1 )


yes
['plane', 'large-vehicle']
yes
['plane']
yes
['plane']
yes
['plane']
yes
['plane']
yes
['plane']
yes
['plane']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['bridge', 'storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['storage-tank']
yes
['stora

In [14]:
pr_ship = prompt_results_df[prompt_results_df['prompt']== 'Question: Is there a ship? Answer:']
pr_ship['correct'] = pr_ship.apply(lambda x: correct_class_found(x,'ship'), axis=1 )

print(f"{pr_ship['correct'].sum()}/{len(pr_ship.index)} {100*pr_ship['correct'].sum()/len(pr_ship.index):.2f}%")

['large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle', 'small-vehicle']
yes
['small-vehicle']
yes
['small-vehicle', 'large-vehicle']
yes
['large-vehicle']
yes
['small-vehicle', 'large-ve

C:\Users\neals\AppData\Local\Temp\ipykernel_32604\1077238267.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_ship['correct'] = pr_ship.apply(lambda x: correct_class_found(x,'ship'), axis=1 )


In [15]:
for id, nam in category_map.items():
    print(f'{id},{nam}')

1,small-vehicle
2,large-vehicle
3,ship
4,harbor
5,baseball-diamond
6,soccer-ball-field
7,ground-track-field
8,swimming-pool
9,roundabout
10,tennis-court
11,basketball-court
12,plane
13,helicopter
14,bridge
15,storage-tank
16,container-crane
